In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, lit
import re

In [2]:
# ==============================
# 1. Inicializar Spark Session
# ==============================
spark = SparkSession.builder \
    .appName("HDFS_NiFi_Data_Cleaning") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/30 17:12:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/30 17:12:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [11]:
# ==============================
# 2. Paths en HDFS
# ==============================
file_familia = "/user/nifi/Resumen_Valores-VENTA_POR_FAMILIA2.csv"
file_producto = "/user/nifi/Resumen_Valores-VENTA_POR_PRODUCTO2.csv"

In [12]:
def rename_month_columns(df):
    """
    Renombra los bloques de columnas por mes con las métricas:
    Venta, TGT, PY24, pct
    """
    meses = ["Enero","Febrero","Marzo","Abril","Mayo","Junio",
             "Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre","YTD_JUL"]
    metricas = ["Venta","TGT","PY24","pct"]

    new_cols = ["Producto_Familia"]  # primera columna fija
    col_idx = 1  # empezamos después de Producto_Familia

    for mes in meses:
        for metrica in metricas:
            if col_idx < len(df.columns):
                new_cols.append(f"{mes}_{metrica}")
                col_idx += 1

    # Si sobran columnas (por ruido en el CSV), las nombramos Extra
    while len(new_cols) < len(df.columns):
        new_cols.append(f"Extra_{len(new_cols)}")

    return df.toDF(*new_cols)

In [13]:
def load_and_clean(path):
    # ============================
    # 1. Leer CSV sin header
    # ============================
    df = spark.read.option("sep", ";").option("header", "false").csv(path)

    # ============================
    # 2. Tomar primera fila como header real
    # ============================
    header_row = df.first()

    # ============================
    # 3. Filtrar esa fila del dataframe
    # ============================
    df = df.filter(col("_c0") != header_row[0])

    # ============================
    # 4. Columnas temporales
    # ============================
    tmp_headers = [f"Col_{i}" for i in range(len(df.columns))]
    df = df.toDF(*tmp_headers)

    # ============================
    # 5. Renombrar columnas a Mes_Métrica
    # ============================
    df = rename_month_columns(df)

    return df

In [14]:
# ==============================
# 4. Aplicar función
# ==============================
df_familia = load_and_clean(file_familia)
df_producto = load_and_clean(file_producto)

In [15]:
print(df_familia)

DataFrame[Producto_Familia: string, Enero_Venta: string, Enero_TGT: string, Enero_PY24: string, Enero_pct: string, Febrero_Venta: string, Febrero_TGT: string, Febrero_PY24: string, Febrero_pct: string, Marzo_Venta: string, Marzo_TGT: string, Marzo_PY24: string, Marzo_pct: string, Abril_Venta: string, Abril_TGT: string, Abril_PY24: string, Abril_pct: string, Mayo_Venta: string, Mayo_TGT: string, Mayo_PY24: string, Mayo_pct: string, Junio_Venta: string, Junio_TGT: string, Junio_PY24: string, Junio_pct: string, Julio_Venta: string, Julio_TGT: string, Julio_PY24: string, Julio_pct: string, Agosto_Venta: string, Agosto_TGT: string, Agosto_PY24: string, Agosto_pct: string, Septiembre_Venta: string, Septiembre_TGT: string, Septiembre_PY24: string, Septiembre_pct: string, Octubre_Venta: string, Octubre_TGT: string, Octubre_PY24: string, Octubre_pct: string, Noviembre_Venta: string, Noviembre_TGT: string, Noviembre_PY24: string, Noviembre_pct: string, Diciembre_Venta: string, Diciembre_TGT: str

In [16]:
print(df_producto)

DataFrame[Producto_Familia: string, Enero_Venta: string, Enero_TGT: string, Enero_PY24: string, Enero_pct: string, Febrero_Venta: string, Febrero_TGT: string, Febrero_PY24: string, Febrero_pct: string, Marzo_Venta: string, Marzo_TGT: string, Marzo_PY24: string, Marzo_pct: string, Abril_Venta: string, Abril_TGT: string, Abril_PY24: string, Abril_pct: string, Mayo_Venta: string, Mayo_TGT: string, Mayo_PY24: string, Mayo_pct: string, Junio_Venta: string, Junio_TGT: string, Junio_PY24: string, Junio_pct: string, Julio_Venta: string, Julio_TGT: string, Julio_PY24: string, Julio_pct: string, Agosto_Venta: string, Agosto_TGT: string, Agosto_PY24: string, Agosto_pct: string, Septiembre_Venta: string, Septiembre_TGT: string, Septiembre_PY24: string, Septiembre_pct: string, Octubre_Venta: string, Octubre_TGT: string, Octubre_PY24: string, Octubre_pct: string, Noviembre_Venta: string, Noviembre_TGT: string, Noviembre_PY24: string, Noviembre_pct: string, Diciembre_Venta: string, Diciembre_TGT: str

In [17]:
df_familia.printSchema()
df_producto.printSchema()

root
 |-- Producto_Familia: string (nullable = true)
 |-- Enero_Venta: string (nullable = true)
 |-- Enero_TGT: string (nullable = true)
 |-- Enero_PY24: string (nullable = true)
 |-- Enero_pct: string (nullable = true)
 |-- Febrero_Venta: string (nullable = true)
 |-- Febrero_TGT: string (nullable = true)
 |-- Febrero_PY24: string (nullable = true)
 |-- Febrero_pct: string (nullable = true)
 |-- Marzo_Venta: string (nullable = true)
 |-- Marzo_TGT: string (nullable = true)
 |-- Marzo_PY24: string (nullable = true)
 |-- Marzo_pct: string (nullable = true)
 |-- Abril_Venta: string (nullable = true)
 |-- Abril_TGT: string (nullable = true)
 |-- Abril_PY24: string (nullable = true)
 |-- Abril_pct: string (nullable = true)
 |-- Mayo_Venta: string (nullable = true)
 |-- Mayo_TGT: string (nullable = true)
 |-- Mayo_PY24: string (nullable = true)
 |-- Mayo_pct: string (nullable = true)
 |-- Junio_Venta: string (nullable = true)
 |-- Junio_TGT: string (nullable = true)
 |-- Junio_PY24: string 

In [18]:
df_familia.head(2)

[Row(Producto_Familia='Producto', Enero_Venta='Venta ', Enero_TGT='TGT ', Enero_PY24='PY 24', Enero_pct='% ', Febrero_Venta='Venta ', Febrero_TGT='TGT ', Febrero_PY24='PY 24', Febrero_pct='% ', Marzo_Venta='Venta ', Marzo_TGT='TGT ', Marzo_PY24='PY 24', Marzo_pct='% ', Abril_Venta='Venta ', Abril_TGT='TGT ', Abril_PY24='PY 24', Abril_pct='% ', Mayo_Venta='Venta ', Mayo_TGT='TGT ', Mayo_PY24='PY 24', Mayo_pct='% ', Junio_Venta='Venta ', Junio_TGT='TGT ', Junio_PY24='PY 24', Junio_pct='% ', Julio_Venta='Venta ', Julio_TGT='TGT ', Julio_PY24='PY 24', Julio_pct='% ', Agosto_Venta='Venta ', Agosto_TGT='TGT ', Agosto_PY24='PY 24', Agosto_pct='% ', Septiembre_Venta='Venta ', Septiembre_TGT='TGT ', Septiembre_PY24='PY 24', Septiembre_pct='% ', Octubre_Venta='Venta ', Octubre_TGT='TGT ', Octubre_PY24='PY 24', Octubre_pct='% ', Noviembre_Venta='Venta ', Noviembre_TGT='TGT ', Noviembre_PY24='PY 24', Noviembre_pct='% ', Diciembre_Venta='Venta ', Diciembre_TGT='TGT ', Diciembre_PY24='PY 24', Diciem

In [19]:
df_producto.head(2)

[Row(Producto_Familia='Producto', Enero_Venta='Venta ', Enero_TGT='TGT ', Enero_PY24='PY 24', Enero_pct='% ', Febrero_Venta='Venta ', Febrero_TGT='TGT ', Febrero_PY24='PY 24', Febrero_pct='% ', Marzo_Venta='Venta ', Marzo_TGT='TGT ', Marzo_PY24='PY 24', Marzo_pct='% ', Abril_Venta='Venta ', Abril_TGT='TGT ', Abril_PY24='PY 24', Abril_pct='% ', Mayo_Venta='Venta ', Mayo_TGT='TGT ', Mayo_PY24='PY 24', Mayo_pct='% ', Junio_Venta='Venta ', Junio_TGT='TGT ', Junio_PY24='PY 24', Junio_pct='% ', Julio_Venta='Venta ', Julio_TGT='TGT ', Julio_PY24='PY 24', Julio_pct='% ', Agosto_Venta='Venta ', Agosto_TGT='TGT ', Agosto_PY24='PY 24', Agosto_pct='% ', Septiembre_Venta='Venta ', Septiembre_TGT='TGT ', Septiembre_PY24='PY 24', Septiembre_pct='% ', Octubre_Venta='Venta ', Octubre_TGT='TGT ', Octubre_PY24='PY 24', Octubre_pct='% ', Noviembre_Venta='Venta ', Noviembre_TGT='TGT ', Noviembre_PY24='PY 24', Noviembre_pct='% ', Diciembre_Venta='Venta ', Diciembre_TGT='TGT ', Diciembre_PY24='PY 24', Diciem

In [20]:
# De forma alargada
from pyspark.sql.functions import expr, regexp_extract, first
from pyspark.sql import functions as F

def unpivot_to_wide(df):
    id_col = "Producto_Familia"
    value_cols = [c for c in df.columns if c != id_col]

    # Crear expresión stack dinámicamente
    expr_str = "stack({0}, {1}) as (Columna, Valor)".format(
        len(value_cols),
        ",".join([f"'{c}', {c}" for c in value_cols])
    )

    # Expandir a formato largo
    df_long = df.select(id_col, expr(expr_str))

    # Separar Mes y Métrica
    df_long = df_long.withColumn("Mes", regexp_extract("Columna", r"^(.*)_(Venta|TGT|PY24|pct)$", 1)) \
                     .withColumn("Metrica", regexp_extract("Columna", r"^(.*)_(Venta|TGT|PY24|pct)$", 2)) \
                     .drop("Columna")

    # Pivotear: convertir Métrica en columnas
    df_wide = df_long.groupBy(id_col, "Mes").pivot("Metrica").agg(F.first("Valor"))

    return df_wide


In [21]:
from pyspark.sql.functions import col

# Quitar la fila "Producto"
df_familia_clean = df_familia.filter(df_familia["Producto_Familia"] != "Producto")
df_producto_clean = df_producto.filter(df_producto["Producto_Familia"] != "Producto")

# Aplicar unpivot corregido (con pivot final)
df_familia_wide = unpivot_to_wide(df_familia_clean)
df_producto_wide = unpivot_to_wide(df_producto_clean)

# Convertir columnas numéricas a double
num_cols = ["Venta", "TGT", "PY24", "pct"]

for c in num_cols:
    df_familia_wide = df_familia_wide.withColumn(c, col(c).cast("double"))
    df_producto_wide = df_producto_wide.withColumn(c, col(c).cast("double"))

# Mostrar resultados
print("=== Familia ===")
df_familia_wide.show(20, truncate=False)

print("=== Producto ===")
df_producto_wide.show(20, truncate=False)


=== Familia ===
+----------------+----------+---------+-----------+---------+-----------+
|Producto_Familia|Mes       |PY24     |TGT        |Venta    |pct        |
+----------------+----------+---------+-----------+---------+-----------+
|AGGLAD          |Abril     |24635.46 |120891.2407|91519.9  |0.757043269|
|AGGLAD          |Agosto    |25769.42 |2560.845455|0.0      |0.0        |
|AGGLAD          |Diciembre |71802.0  |2681.545455|0.0      |0.0        |
|AGGLAD          |Enero     |22534.35 |110094.81  |110094.81|1.0        |
|AGGLAD          |Febrero   |31785.06 |114546.1364|110784.62|0.96716156 |
|AGGLAD          |Julio     |29784.66 |133581.4492|18240.65 |0.136550772|
|AGGLAD          |Junio     |20271.95 |120891.2407|92445.3  |0.764698083|
|AGGLAD          |Marzo     |31785.06 |114546.1364|81102.53 |0.708033745|
|AGGLAD          |Mayo      |26193.67 |120891.2407|80999.82 |0.670022241|
|AGGLAD          |Noviembre |52414.29 |2681.545455|0.0      |0.0        |
|AGGLAD          |Octu

In [23]:
from pyspark.sql.functions import trim, upper

# Normalizar columna Mes en ambos dataframes
df_familia_wide = df_familia_wide.withColumn("Mes", trim(upper(df_familia_wide["Mes"])))
df_producto_wide = df_producto_wide.withColumn("Mes", trim(upper(df_producto_wide["Mes"])))


In [24]:
from pyspark.sql.functions import when, col

# Lista de métricas a convertir
metricas = ["Venta", "TGT", "PY24", "pct"]

# Familia
for m in metricas:
    df_familia_wide = df_familia_wide.withColumn(
        m,
        when(col(m).isNull(), 0.0).otherwise(col(m).cast("double"))
    )

# Producto
for m in metricas:
    df_producto_wide = df_producto_wide.withColumn(
        m,
        when(col(m).isNull(), 0.0).otherwise(col(m).cast("double"))
    )


In [25]:
from pyspark.sql.functions import lit

# Agregar columna de nivel
df_familia_wide = df_familia_wide.withColumn("Nivel", lit("Familia"))
df_producto_wide = df_producto_wide.withColumn("Nivel", lit("Producto"))

# Unir ambos datasets
df_total = (
    df_familia_wide
    .unionByName(df_producto_wide, allowMissingColumns=True)
)


In [26]:
df_total.show(100, truncate=False)

+----------------+----------+----------+-----------+---------+-----------+-------+
|Producto_Familia|Mes       |PY24      |TGT        |Venta    |pct        |Nivel  |
+----------------+----------+----------+-----------+---------+-----------+-------+
|AGGLAD          |ABRIL     |24635.46  |120891.2407|91519.9  |0.757043269|Familia|
|AGGLAD          |AGOSTO    |25769.42  |2560.845455|0.0      |0.0        |Familia|
|AGGLAD          |DICIEMBRE |71802.0   |2681.545455|0.0      |0.0        |Familia|
|AGGLAD          |ENERO     |22534.35  |110094.81  |110094.81|1.0        |Familia|
|AGGLAD          |FEBRERO   |31785.06  |114546.1364|110784.62|0.96716156 |Familia|
|AGGLAD          |JULIO     |29784.66  |133581.4492|18240.65 |0.136550772|Familia|
|AGGLAD          |JUNIO     |20271.95  |120891.2407|92445.3  |0.764698083|Familia|
|AGGLAD          |MARZO     |31785.06  |114546.1364|81102.53 |0.708033745|Familia|
|AGGLAD          |MAYO      |26193.67  |120891.2407|80999.82 |0.670022241|Familia|
|AGG